In [8]:
pip install diffprivlib

Looking in indexes: http://build-nexus.prod.crto.in/repository/pypi/simple
You should consider upgrading via the '/home/a.gilotte/.local/share/jupyter/runtime/kernel-venvs/f8a52dae-0bc1-4ad2-a486-a82df16021fc/env/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [9]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("../src")
from myimports  import *
import myJupyterUtils ## Remove stacktraces on Keyboardinterupt
plt.style.use('ggplot')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
from featuremappings import AggDataset
from basicmodels import LogisticModel, NaiveBayesModel, LogisticModelWithCF
from validation import MetricsComputer,  LLH


In [11]:
from RandomProjectedPrivateLogistic import RPPrivateLogistic
from basicmodels import PrivateLogistic

In [12]:
from aggLogistic import AggLogistic
from agg_mrf_model import AggMRFModel
import Optimizers

In [13]:
Validation = MetricsComputer("click")

## Download Data

In [14]:
import loaddata
train, valid, allvars = loaddata.run()
## features def: https://confluence.criteois.com/pages/viewpage.action?spaceKey=RSC&title=Dataset+06+%3A+Attribution+Modeling+for+Bidding


164545
click 0.3607523777689994
conversion 0.04847306208028199
attribution 0.026588471238870825
campaign 672
click_nb 84
time_since_last_click 8
cat1 9
cat2 59
cat3 1684
cat4 15
cat5 51
cat6 30
cat7 10933
cat8 11
cat9 30


In [15]:
def addToResults( model, name , epsilon, regulL2 , otherparams = {}  ):
    dico = otherparams.copy()
    dico["model"] = name
    dico["valid"] = Validation.getLLH( model, valid )
    dico["train"] = Validation.getLLH( model, train )
    dico["epsilon"] = epsilon
    dico["regul"] = regulL2
    dico["features"] =  ",".join( model. features)
    res = pd.DataFrame( [dico] )
    print(res)
    return pd.concat( [ results , res ] )

In [16]:
results = pd.DataFrame()

# Data set with limited number of features

In [ ]:
features = ['cat1', 'cat2', 'cat8', 'cat9' ]
aggdata_nonoise = AggDataset( features, "*&*", train )
epsilons = [ 0.1 ,1.0, 10.0 ]
aggdata_noised = {}
for epsilon0 in epsilons:
    aggdata_noised[epsilon0] = AggDataset( features, "*&*", train , epsilon0=epsilon0)


In [ ]:
logistic.features

##### logistic Regression
- Requires full logs, not privacy preserving.
- "Skyline"

In [ ]:
regulL2s = [ 1.0,  4.0 , 16.0 , 64.0, 256 ]
for regulL2 in regulL2s:
    logistic = AggLogistic(  aggdata_nonoise , features, clicksCfs = "*&*" , regulL2=regulL2 )
    logistic.fit(train [features] , nbIter=500)
    results = addToResults( logistic, "logistic", 0, regulL2  )

In [ ]:
results

##### Private logistic regression
- From ibm diffprivlib
- Adding some noise to the loss
- Does not scales well with input vector dimensions, therefore using random projections to limit dimensionality

In [ ]:
%%time
for epsilon in epsilons:
    aggdata = aggdata_noised[epsilon]
    regulL2s = [ 1.0,  4.0 , 16.0 , 64.0, 256, 1024 , 1024*4 ]    
    for l2 in regulL2s:
        for p in [ 10,30,100,300, 1000 ]:
            logistic = RPPrivateLogistic("click" , features, "*&*",  train, lambdaL2=l2,epsilon= epsilon,
                                     projectionSize=p )
            logistic.fit(train)
            results = addToResults( logistic, "privatelogistic",  epsilon , l2, {"dimensions":p}  )

##### Models learned from agg data only

In [ ]:
%%time
for epsilon in epsilons:
    aggdata = aggdata_noised[epsilon]
    for l2 in [  4.0 , 16.0 , 64.0, 256, 1024 , 1024*4 ]:
        self = AggMRFModel( aggdata, features , 
                       exactComputation=False , clicksCfs = "*&*", displaysCfs="*&*",
                      nbSamples = 50000, regulL2=1.0 , regulL2Click = l2,
                        laplaceEpsilon = aggdata.epsilon )        
        self.fit(200)
        results = addToResults( self, "aggmodel",  epsilon , l2  )

In [ ]:
results[results.epsilon == 1.0].sort_values( "valid" )

In [ ]:
a = results[(results.model == "aggmodel") & (results.epsilon == 0.1) &(results.train>0)]
a

In [ ]:
a = results[results.model == "logistic"]
plt.plot( a.train.values , a.valid.values  , "x" )


a = results[(results.model == "aggmodel") & (results.epsilon == 0.1) &(results.train>0)]
plt.plot( a.train.values , a.valid.values  , "x" )

a = results[(results.model == "privatelogistic") & (results.epsilon == 0.1) &(results.train>0)]
plt.plot( a.train.values , a.valid.values  , "kx" )



## Big dataset (11 features)

In [18]:
features =['campaign', 'time_since_last_click', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9']
aggdata_nonoise = AggDataset( features, "*&*", train )
epsilons = [ 1.0, 10.0 ]

In [ ]:
aggdata_noised = {}
for epsilon0 in epsilons:
    aggdata_noised[epsilon0] = AggDataset( features, "*&*", train , epsilon0=epsilon0)

In [13]:
regulL2s = [  40.0 , 160.0 , 640.0, 2560 ]
for regulL2 in regulL2s:
    logistic = AggLogistic(  aggdata_nonoise , features, clicksCfs = "*&*" , regulL2=regulL2 )
    logistic.fit(train [features] , nbIter=50)
    results = addToResults( logistic, "logistic", 0, regulL2  )

      model   valid     train  epsilon  regul  \
0  logistic  0.0727  0.135455        0   40.0   

                                            features  
0  campaign,time_since_last_click,cat1,cat2,cat3,...  
      model     valid     train  epsilon  regul  \
0  logistic  0.072766  0.098063        0  160.0   

                                            features  
0  campaign,time_since_last_click,cat1,cat2,cat3,...  
      model     valid     train  epsilon  regul  \
0  logistic  0.065278  0.074811        0  640.0   

                                            features  
0  campaign,time_since_last_click,cat1,cat2,cat3,...  
      model     valid     train  epsilon  regul  \
0  logistic  0.049948  0.053427        0   2560   

                                            features  
0  campaign,time_since_last_click,cat1,cat2,cat3,...  


In [12]:
%%time
for epsilon in epsilons:
    regulL2s = [ 160.0 , 640.0, 2560, 2560*4 , 2560*16 ]    
    for l2 in regulL2s:
        for p in [ 30,100,300, 1000, 3000 ]:
            print( epsilon, l2,p)
            logistic = RPPrivateLogistic("click" , features, "*&*",  train, lambdaL2=l2,epsilon= epsilon,
                                     projectionSize=p )
            logistic.fit(train)
            results = addToResults( logistic, "privatelogistic",  epsilon , l2, {"dimensions":p}  )

1.0 160.0 30
   dimensions            model     valid     train  epsilon  regul  \
0          30  privatelogistic  0.014299  0.015454      1.0  160.0   

                                            features  
0  campaign,time_since_last_click,cat1,cat2,cat3,...  
1.0 160.0 100
   dimensions            model     valid     train  epsilon  regul  \
0         100  privatelogistic  0.017626  0.017337      1.0  160.0   

                                            features  
0  campaign,time_since_last_click,cat1,cat2,cat3,...  
1.0 160.0 300
   dimensions            model    valid     train  epsilon  regul  \
0         300  privatelogistic  0.02077  0.021447      1.0  160.0   

                                            features  
0  campaign,time_since_last_click,cat1,cat2,cat3,...  
1.0 160.0 1000
   dimensions            model     valid     train  epsilon  regul  \
0        1000  privatelogistic  0.019386  0.018945      1.0  160.0   

                                            features

   dimensions            model     valid     train  epsilon  regul  \
0         100  privatelogistic  0.009256  0.009746     10.0  640.0   

                                            features  
0  campaign,time_since_last_click,cat1,cat2,cat3,...  
10.0 640.0 300
   dimensions            model     valid     train  epsilon  regul  \
0         300  privatelogistic  0.011157  0.011606     10.0  640.0   

                                            features  
0  campaign,time_since_last_click,cat1,cat2,cat3,...  
10.0 640.0 1000
   dimensions            model    valid     train  epsilon  regul  \
0        1000  privatelogistic  0.01236  0.012532     10.0  640.0   

                                            features  
0  campaign,time_since_last_click,cat1,cat2,cat3,...  
10.0 640.0 3000
   dimensions            model     valid     train  epsilon  regul  \
0        3000  privatelogistic  0.012886  0.013129     10.0  640.0   

                                            features  
0  cam

In [15]:
%%time
for epsilon in epsilons:
    regulL2s = [ 40.0, 10.0  ]    
    for l2 in regulL2s:
        for p in [ 3000,1000,300, 100, 30 ]:
            print( epsilon, l2,p)
            logistic = RPPrivateLogistic("click" , features, "*&*",  train, lambdaL2=l2,epsilon= epsilon,
                                     projectionSize=p )
            logistic.fit(train)
            results = addToResults( logistic, "privatelogistic",  epsilon , l2, {"dimensions":p}  )

1.0 40.0 3000
   dimensions            model     valid     train  epsilon  regul  \
0        3000  privatelogistic -0.248148 -0.240398      1.0   40.0   

                                            features  
0  campaign,time_since_last_click,cat1,cat2,cat3,...  
1.0 40.0 1000
   dimensions            model     valid     train  epsilon  regul  \
0        1000  privatelogistic -0.030128 -0.026317      1.0   40.0   

                                            features  
0  campaign,time_since_last_click,cat1,cat2,cat3,...  
1.0 40.0 300
   dimensions            model     valid    train  epsilon  regul  \
0         300  privatelogistic  0.018916  0.02209      1.0   40.0   

                                            features  
0  campaign,time_since_last_click,cat1,cat2,cat3,...  
1.0 40.0 100
   dimensions            model     valid     train  epsilon  regul  \
0         100  privatelogistic  0.022841  0.023902      1.0   40.0   

                                            features  

In [19]:
results.query( 'epsilon==10.0' ) .sort_values('valid')

,dimensions,model,valid,train,epsilon,regul,features
0,1000,privatelogistic,0.000255,0.000295,10.0,40960.0,"campaign,time_since_last_click,cat1,cat2,cat3,..."
0,3000,privatelogistic,0.000278,0.000315,10.0,40960.0,"campaign,time_since_last_click,cat1,cat2,cat3,..."
0,300,privatelogistic,0.000285,0.000321,10.0,40960.0,"campaign,time_since_last_click,cat1,cat2,cat3,..."
0,100,privatelogistic,0.000307,0.000356,10.0,40960.0,"campaign,time_since_last_click,cat1,cat2,cat3,..."
0,30,privatelogistic,0.000380,0.000425,10.0,40960.0,"campaign,time_since_last_click,cat1,cat2,cat3,..."
0,30,privatelogistic,0.000790,0.000881,10.0,10240.0,"campaign,time_since_last_click,cat1,cat2,cat3,..."
0,100,privatelogistic,0.000952,0.001002,10.0,10240.0,"campaign,time_since_last_click,cat1,cat2,cat3,..."
0,3000,privatelogistic,0.001123,0.001174,10.0,10240.0,"campaign,time_since_last_click,cat1,cat2,cat3,..."
0,1000,privatelogistic,0.001149,0.001203,10.0,10240.0,"campaign,time_since_last_click,cat1,cat2,cat3,..."
0,300,privatelogistic,0.001158,0.001205,10.0,10240.0,"campaign,time_since_last_click,cat1,cat2,cat3,..."


In [22]:
%%time
for epsilon in epsilons:
    aggdata = AggDataset( features, "*&*", train , epsilon0=epsilon)
    for l2 in [ 160.0 , 640.0, 2560, 2560*4 , 2560*16 ]:
        self = AggMRFModel( aggdata, features , 
                       exactComputation=False , clicksCfs = "*&*", displaysCfs="*&*",
                      nbSamples = 50000, regulL2=1.0 , regulL2Click = l2,
                        noiseDistribution = aggdata.mechanism.getNoise()  )        
        self.fit(200)
        results = addToResults( self, "aggmodel",  epsilon , l2  )

llh:1.0E+03 a:9.7E-01(1), n:200, g:5.1E+03  --
      model     valid     train  epsilon  regul  \
0  aggmodel  0.045305  0.046101      1.0  160.0   

                                            features  
0  campaign,time_since_last_click,cat1,cat2,cat3,...  
llh:1.0E+03 a:8.3E-01(1), n:200, g:5.5E+03  --
      model     valid    train  epsilon  regul  \
0  aggmodel  0.030115  0.02999      1.0  640.0   

                                            features  
0  campaign,time_since_last_click,cat1,cat2,cat3,...  


KeyboardInterrupt


In [29]:
%%time
for epsilon in epsilons[1:]:
    aggdata = AggDataset( features, "*&*", train , epsilon0=epsilon)
    for l2 in [ 160.0 , 640.0] :# , 2560, 2560*4 , 2560*16 ]:
        self = AggMRFModel( aggdata, features , 
                       exactComputation=False , clicksCfs = "*&*", displaysCfs="*&*",
                      nbSamples = 50000, regulL2=1.0 , regulL2Click = l2,
                        noiseDistribution = aggdata.mechanism.getNoise()  )        
        self.fit(200)
        results = addToResults( self, "aggmodel",  epsilon , l2  )

llh:9.4E+01 a:9.3E-02(1), n:200, g:3.8E+04  --
      model     valid     train  epsilon  regul  \
0  aggmodel  0.059601  0.069238     10.0  160.0   

                                            features  
0  campaign,time_since_last_click,cat1,cat2,cat3,...  
llh:9.4E+01 a:1.1E-01(1), n:200, g:3.2E+04  --
      model     valid     train  epsilon  regul  \
0  aggmodel  0.060704  0.063128     10.0  640.0   

                                            features  
0  campaign,time_since_last_click,cat1,cat2,cat3,...  
CPU times: user 5h 22min 46s, sys: 48min 12s, total: 6h 10min 59s
Wall time: 14h 5min 6s


In [ ]:
%%time
## GAUSSIAN noise
for delta in [ 0.000001 , 0.001]:
    for epsilon in [10.0, 1.0]:
        for l2 in [ 160.0 , 640.0] :# , 2560, 2560*4 , 2560*16 ]:
            aggdata = AggDataset( features, "*&*", train , epsilon0=epsilon ,delta = delta)
            self = AggMRFModel( aggdata, features , 
                           exactComputation=False , clicksCfs = "*&*", displaysCfs="*&*",
                          nbSamples = 50000, regulL2=1.0 , regulL2Click = l2,
                            noiseDistribution = aggdata.mechanism.getNoise()  )        
            self.fit(200)
            results = addToResults( self, "aggmodel",  epsilon , l2 ,  {"delta":delta}  )

GaussianMechanism epsilon:10.0 delta:1e-06 sigma:4.74584727252426
llh:0.0E+00 a:9.9E-02(1), n:200, g:6.4E+03  --
      delta     model     valid     train  epsilon  regul  \
0  0.000001  aggmodel  0.070056  0.080921     10.0  160.0   

                                            features  
0  campaign,time_since_last_click,cat1,cat2,cat3,...  
GaussianMechanism epsilon:10.0 delta:1e-06 sigma:4.74584727252426


1

In [ ]:
results[ (results.model=="privatelogistic") &(results.epsilon==0.1) ].sort_values('valid')

In [28]:
results[results.epsilon==1.0].sort_values('valid')

,dimensions,model,valid,train,epsilon,regul,features
0,3000.0,privatelogistic,-2.102479,-2.083905,1.0,10.0,"campaign,time_since_last_click,cat1,cat2,cat3,..."
0,1000.0,privatelogistic,-0.642570,-0.625136,1.0,10.0,"campaign,time_since_last_click,cat1,cat2,cat3,..."
0,3000.0,privatelogistic,-0.248148,-0.240398,1.0,40.0,"campaign,time_since_last_click,cat1,cat2,cat3,..."
0,1000.0,privatelogistic,-0.030128,-0.026317,1.0,40.0,"campaign,time_since_last_click,cat1,cat2,cat3,..."
0,300.0,privatelogistic,-0.011871,-0.013132,1.0,10.0,"campaign,time_since_last_click,cat1,cat2,cat3,..."
0,3000.0,privatelogistic,-0.000366,0.001727,1.0,160.0,"campaign,time_since_last_click,cat1,cat2,cat3,..."
0,1000.0,privatelogistic,0.000239,0.000296,1.0,40960.0,"campaign,time_since_last_click,cat1,cat2,cat3,..."
0,100.0,privatelogistic,0.000268,0.000313,1.0,40960.0,"campaign,time_since_last_click,cat1,cat2,cat3,..."
0,3000.0,privatelogistic,0.000287,0.000293,1.0,40960.0,"campaign,time_since_last_click,cat1,cat2,cat3,..."
0,300.0,privatelogistic,0.000293,0.000327,1.0,40960.0,"campaign,time_since_last_click,cat1,cat2,cat3,..."


In [30]:
results[results.epsilon==10.0].sort_values('valid')

,dimensions,model,valid,train,epsilon,regul,features
0,1000.0,privatelogistic,0.000255,0.000295,10.0,40960.0,"campaign,time_since_last_click,cat1,cat2,cat3,..."
0,3000.0,privatelogistic,0.000278,0.000315,10.0,40960.0,"campaign,time_since_last_click,cat1,cat2,cat3,..."
0,300.0,privatelogistic,0.000285,0.000321,10.0,40960.0,"campaign,time_since_last_click,cat1,cat2,cat3,..."
0,100.0,privatelogistic,0.000307,0.000356,10.0,40960.0,"campaign,time_since_last_click,cat1,cat2,cat3,..."
0,30.0,privatelogistic,0.000380,0.000425,10.0,40960.0,"campaign,time_since_last_click,cat1,cat2,cat3,..."
0,30.0,privatelogistic,0.000790,0.000881,10.0,10240.0,"campaign,time_since_last_click,cat1,cat2,cat3,..."
0,100.0,privatelogistic,0.000952,0.001002,10.0,10240.0,"campaign,time_since_last_click,cat1,cat2,cat3,..."
0,3000.0,privatelogistic,0.001123,0.001174,10.0,10240.0,"campaign,time_since_last_click,cat1,cat2,cat3,..."
0,1000.0,privatelogistic,0.001149,0.001203,10.0,10240.0,"campaign,time_since_last_click,cat1,cat2,cat3,..."
0,300.0,privatelogistic,0.001158,0.001205,10.0,10240.0,"campaign,time_since_last_click,cat1,cat2,cat3,..."
